In [1]:
import pandas as pd

In [4]:
jan = pd.read_parquet('fhv_tripdata_2021-01.parquet')
feb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [5]:
jan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [7]:
jan.shape

(1154112, 7)

In [8]:
pd.to_datetime(jan['pickup_datetime'])

0         2021-01-01 00:27:00
1         2021-01-01 00:50:00
2         2021-01-01 00:01:00
3         2021-01-01 00:13:09
4         2021-01-01 00:38:31
                  ...        
1154107   2021-01-31 23:43:03
1154108   2021-01-31 23:50:27
1154109   2021-01-31 23:13:46
1154110   2021-01-31 23:58:03
1154111   2021-01-31 23:39:00
Name: pickup_datetime, Length: 1154112, dtype: datetime64[ns]